## Krok 1: Utworzenie połączenia (Connection)

Najpierw definiujemy bezpieczne połączenie do zewnętrznej bazy. Wymaga to uprawnień administratora lub odpowiednich grantów.

> **Uwaga**: W środowisku szkoleniowym to polecenie może wymagać uprawnień, których nie posiadasz. Poniżej przykład dla SQL Server.

In [ ]:
-- Tworzenie połączenia do SQL Server
CREATE CONNECTION IF NOT EXISTS sql_server_connection
TYPE SQLSERVER
OPTIONS (
  host 'sql-server-host.database.windows.net',
  port '1433',
  user 'admin_user',
  password 'secret_password'
);

-- Weryfikacja połączenia
DESCRIBE CONNECTION sql_server_connection;

## Krok 2: Utworzenie Foreign Catalog

Foreign Catalog to "lustro" zewnętrznej bazy danych w Unity Catalog. Widzisz tabele z SQL Servera tak, jakby były tabelami w Databricks.

In [ ]:
-- Mapowanie zewnętrznej bazy 'SalesDB' na katalog w Databricks
CREATE FOREIGN CATALOG IF NOT EXISTS external_sales_system
USING CONNECTION sql_server_connection
OPTIONS (database 'SalesDB');

-- Teraz możemy przeglądać tabele jak lokalne!
SHOW SCHEMAS IN external_sales_system;
SHOW TABLES IN external_sales_system.dbo;

## Krok 3: Efekt "Wow" - Łączenie danych Delta z SQL Server

Łączymy dane historyczne/analityczne z Databricks (Delta Lake) z danymi operacyjnymi z SQL Server (Live) w jednym zapytaniu.

In [ ]:
-- Scenariusz: Analiza sprzedaży (Databricks) wzbogacona o aktualne stany magazynowe (SQL Server)

SELECT 
    p.product_name,
    p.category,
    SUM(s.total_amount) as total_sales_value,
    inv.QuantityOnHand as current_stock_sql_server,
    CASE 
        WHEN inv.QuantityOnHand < 10 THEN 'CRITICAL LOW'
        WHEN inv.QuantityOnHand < 50 THEN 'ORDER SOON'
        ELSE 'OK'
    END as stock_status
FROM 
    workshop.silver.sales s                          -- Tabela Delta (Lokalna)
JOIN 
    workshop.silver.products p                       -- Tabela Delta (Lokalna)
    ON s.product_id = p.product_id
JOIN 
    external_sales_system.dbo.Inventory inv          -- Tabela SQL Server (Zdalna - Live!)
    ON p.product_sku = inv.SKU
GROUP BY 
    p.product_name, 
    p.category, 
    inv.QuantityOnHand
ORDER BY 
    total_sales_value DESC;

### Dlaczego to jest "Wow"?

1.  **Brak ETL**: Nie musieliśmy pisać pipeline'u kopiującego tabelę `Inventory` z SQL Servera.
2.  **Real-time**: Widzimy stan magazynowy dokładnie z momentu wykonania zapytania.
3.  **Prostota**: Używamy standardowego SQL, łącząc systemy jak zwykłe tabele.